In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-25 22:55:57--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt'

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2024-08-25 22:55:58 (9.11 MB/s) - 'input.txt' saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Below this can be changed since we are just using encode and decode and on a character level for encoding and decoding.

In [4]:
stoi = { ch:i for i,ch in enumerate(chars) } # mapping characters to its index
itos = { i:ch for i,ch in enumerate(chars) } # mapping index to its characters
def encode(s):
  res = []
  for c in s:
    res.append(stoi[c])
  return res
def decode(encodedList):
  res = ''
  for i in encodedList:
    res += itos[i]
  return res

Example below of how to use encode and decode string

In [5]:
encodedList = encode("Hello World")
print(encodedList)
decodedString = decode(encodedList)
print(decodedString)

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
Hello World


In [6]:
# encode the entire dataset

print(encode(text)[:100])

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43, 43, 42, 1, 39, 52, 63, 1, 44, 59, 56, 58, 46, 43, 56, 6, 1, 46, 43, 39, 56, 1, 51, 43, 1, 57, 54, 43, 39, 49, 8, 0, 0, 13, 50, 50, 10, 0, 31, 54, 43, 39, 49, 6, 1, 57, 54, 43, 39, 49, 8, 0, 0, 18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 37, 53, 59]


In [ ]:
import tensorflow as tf

In [21]:
# encode my text as a tensor
tensor_data = tf.Variable(encode(text), dtype=tf.int64)
print(tensor_data.shape)
print(tensor_data.dtype)
print(tensor_data)

(1115394,)
<dtype: 'int64'>
<tf.Variable 'Variable:0' shape=(1115394,) dtype=int64, numpy=array([18, 47, 56, ..., 45,  8,  0])>


In [28]:
length_train = int(0.9 * tensor_data.shape[0])

training_data = tensor_data[:length_train]
validation_data = tensor_data[length_train:]

print(training_data)
print(validation_data)

tf.Tensor([18 47 56 ... 43 56 43], shape=(1003854,), dtype=int64)
tf.Tensor([12  0  0 ... 45  8  0], shape=(111540,), dtype=int64)


In [30]:
# creating a block size
block_size = 8

input_data = training_data[:block_size]
output_data = training_data[1:block_size+1]

for i in range(8):
    context = input_data[:i+1]
    target = output_data[i]
    print(f"Input: {context}, Output: {target}")

Input: [18], Output: 47
Input: [18 47], Output: 56
Input: [18 47 56], Output: 57
Input: [18 47 56 57], Output: 58
Input: [18 47 56 57 58], Output: 1
Input: [18 47 56 57 58  1], Output: 15
Input: [18 47 56 57 58  1 15], Output: 47
Input: [18 47 56 57 58  1 15 47], Output: 58


In [41]:
# creating batches to reduce computation on cpu
tf.random.set_seed(1337)

batch_size = 4 # number of independent sequences we will process in parallel
block_size = 8 # maximum context for predictions

def get_batch(data_type):
    data = None
    if data_type == "Training": data = training_data
    else: data = validation_data

    max_start = tf.size(data) - block_size

    random_indices = tf.random.uniform(shape=[batch_size], minval=0, maxval=max_start, dtype=tf.int32)

    # make starting indices new matrices for context
    # making new matrices for target
    x = []
    y = []

    for i in random_indices:
        x.append(data[i:i+block_size])
        y.append(data[i+1:i+1+block_size])
    
    x = tf.stack(x)
    y = tf.stack(y)
    return x, y
    print(f"Context Matrix: {x}")
    print(f"Target Matrix: {y}")

x_batch, y_batch = get_batch("Training")

print(f"Context Matrix: {x_batch}")
print(f"Target Matrix: {y_batch}")

Context Matrix: [[ 1 51 63  1 41 53 39 58]
 [39 42  0 20 47 57  1 52]
 [32 53  1 56 43 60 43 50]
 [54 39 52 63  1 54 47 43]]
Target Matrix: [[51 63  1 41 53 39 58  6]
 [42  0 20 47 57  1 52 39]
 [53  1 56 43 60 43 50  1]
 [39 52 63  1 54 47 43 41]]


In [43]:
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = x_batch[b, :t+1]
        target = y_batch[b,t]
        print(f"when input is {context.numpy().tolist()} the target: {target}")

when input is [1] the target: 51
when input is [1, 51] the target: 63
when input is [1, 51, 63] the target: 1
when input is [1, 51, 63, 1] the target: 41
when input is [1, 51, 63, 1, 41] the target: 53
when input is [1, 51, 63, 1, 41, 53] the target: 39
when input is [1, 51, 63, 1, 41, 53, 39] the target: 58
when input is [1, 51, 63, 1, 41, 53, 39, 58] the target: 6
when input is [39] the target: 42
when input is [39, 42] the target: 0
when input is [39, 42, 0] the target: 20
when input is [39, 42, 0, 20] the target: 47
when input is [39, 42, 0, 20, 47] the target: 57
when input is [39, 42, 0, 20, 47, 57] the target: 1
when input is [39, 42, 0, 20, 47, 57, 1] the target: 52
when input is [39, 42, 0, 20, 47, 57, 1, 52] the target: 39
when input is [32] the target: 53
when input is [32, 53] the target: 1
when input is [32, 53, 1] the target: 56
when input is [32, 53, 1, 56] the target: 43
when input is [32, 53, 1, 56, 43] the target: 60
when input is [32, 53, 1, 56, 43, 60] the target: 4

In [ ]:
# create a neural network: example uses bigram we can use something else...